# Polytope example notebook - Pressure level data from High Resolution model

This notebook will show you basic use of Polytope client.  
It should be used as skeleton example for building Polytope API requests.

First we import Polytope Client and xarray library to inspect the data.

In [ ]:
from polytope.api import Client
import xarray as xr

In order to use the Client, we need to start it.  
We can then list available collections. In HiDALGO Project users have the option to use 'hidalgo-mars' collection.

In [ ]:
c = Client()

In [ ]:
c.list_collections()

### Get the temperature data on pressure levels
This example will get us data on all 13 pressure levels, from 00 model run, first 12 steps from each run (0 to 11), for area around Hungary and at 0.5 by 0.5 degree resolution.  

To build efficient Polytope request for data and spend least time in the queue, it is recomended that each request access only one tape with data.   
For the data on Pressure Levels data from one forecast run (one date and one time) are on one tape. This means that it is recomended to have a separate request for each time and each date.

Pressure Level data is interpolated into significant meteorological pressure levels (1/2/3/5/7/10/20/30/50/70/100/150/200/250/300/400/500/600/700/800/850/900/925/950/1000). 

When you are trying out requests, try coarser resolution until you're happy, so you get smaller files, and when you're happy, increase it to 0.1/0.1.

In [ ]:
request_1 = {
    'stream': 'oper',
    'levtype': 'pl',
    'levelist': '1000/950/850/700/500/400/300/250/150/100/50',
    'param': 't',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

In [ ]:
c.retrieve('hidalgo-mars', request_1, 'pl_t.grib')

### Submitting a list of requests
Sometimes we want to split our request into 2 smaller requests. We can then submit both requests together as a list.  
Let's make a request that accesses the same set of data from the same date, just run from 12 UTC.

In [ ]:
request_2 = {
    'stream': 'oper',
    'levtype': 'pl',
    'levelist': '1000/950/850/700/500/400/300/250/150/100/50',
    'param': 't',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '12',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

Or we can add more requests...

In [ ]:
request_3 = {
    'stream': 'oper',
    'levtype': 'pl',
    'levelist': '1000/950/850/700/500/400/300/250/150/100/50',
    'param': 'q',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}
request_4 = {
    'stream': 'oper',
    'levtype': 'pl',
    'levelist': '1000/950/850/700/500/400/300/250/150/100/50',
    'param': 'q',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '12',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}
request_5 = {
    'stream': 'oper',
    'levtype': 'pl',
    'levelist': '1000/950/850/700/500/400/300/250/150/100/50',
    'param': 'r',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}
request_6 = {
    'stream': 'oper',
    'levtype': 'pl',
    'levelist': '1000/950/850/700/500/400/300/250/150/100/50',
    'param': 'r',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '12',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

These requests can then be submitted together. The result will be downloaded in the same file.  
Note that at the moment one user can have maximum of 6 active requests, and all above that are put in the queue. So it is best not to run more than 6 requests in paralel.

In [ ]:
c.retrieve('hidalgo-mars', [request_1,request_2,request_3,
                            request_4,request_5,request_6], 'pl_20170305.grib')

In [ ]:
ds = xr.open_dataset('pl_20170305.grib',engine='cfgrib')
ds

In [ ]:
ds.t.sel(time='2017-03-05 12:00:00',step=('05:00:00'),isobaricInhPa=1000).plot()